In [11]:
# initial imports
import pandas as pd
import numpy as np
import panel as pn
import hvplot.pandas
from panel.interact import interact
import plotly.express as px

In [20]:
##INNER FUNCTION
def running_backs(rbRushYds, rbRushTds, rbFum):

    #read in CSV
    runningback_path = ("../Resources/updated_rb_2019_data.csv")
    rb_df = pd.read_csv(runningback_path)
    rb_df.set_index('name',inplace = True)
        
    #DATA CLEAN SECTION
    #fill nan with 0's
    rb_df.fillna(0, inplace = True)
    
    #per game metrics
    rb_df['rush_attempts_pg'] = round(rb_df['rush_attempts'] / rb_df['games_played'] , 2)
    rb_df['rush_yds_pg'] = round(rb_df['rush_yards'] / rb_df['games_played'] , 2)
    rb_df['rush_tds_pg'] = round(rb_df['rush_tds'] / rb_df['games_played'] , 2)
    rb_df['receptions_pg'] = round(rb_df['receptions'] / rb_df['games_played'] , 2)
    rb_df['rec_yds_pg'] = round(rb_df['rec_yds'] / rb_df['games_played'] , 2)
    rb_df['rec_tds_pg'] = round(rb_df['rec_tds'] / rb_df['games_played'] , 2)
    rb_df['fum_pg'] = round(rb_df['fumbles'] / rb_df['games_played'] , 2)
    rb_df['fum_lost_pg'] = round(rb_df['fumbles_lost'] / rb_df['games_played'] , 2)

    
    #PARAMATERS SECTION
    rb_df = rb_df.loc[rb_df['rush_yds_pg'] >= rbRushYds[0]]
    rb_df = rb_df.loc[rb_df['rush_yds_pg'] <= rbRushYds[1]]
    rb_df = rb_df.loc[rb_df['rush_tds_pg'] >= rbRushTds[0]]
    rb_df = rb_df.loc[rb_df['rush_tds_pg'] <= rbRushTds[1]]
    rb_df = rb_df.loc[rb_df['fum_pg'] >= rbFum[0]]
    rb_df = rb_df.loc[rb_df['fum_pg'] <= rbFum[1]]
    
    
    #PLOT SECTION
    #Plot bar charts
    rb_rec_rush = rb_df[["rec_yds_pg","rush_yds_pg"]]
    rb_rec_rush_bar = rb_rec_rush.hvplot.bar(width = 1500, height = 300, stacked = True, rot = 90)

    rb_fum = rb_df[["fum_pg","fum_lost_pg"]]
    rb_fum_bar = rb_fum.hvplot.bar(width = 1500, height = 300, shared_axes = False, rot = 90)

    rbrush_by_player = rb_df[["rush_yds_pg","rush_tds_pg"]]
    rb_rush_scatter = rbrush_by_player.hvplot.scatter(x = 'rush_yds_pg', y = 'rush_tds_pg', trendline = "ols")

    rbrec_by_player = rb_df[["rec_yds_pg","rec_tds_pg"]]
    rb_rec_scatter = px.scatter(rbrec_by_player, x = 'rec_yds_pg', y = 'rec_tds_pg', trendline = "ols")



    return pn.Column(pn.Row(rb_rec_rush_bar,rb_fum_bar),
                     pn.Row(rb_rush_scatter,rb_rec_scatter)
                    )

In [21]:
##OUTER FUNCTION PARAMATERS
pn.interact(running_backs,
       rbRushYds = pn.widgets.RangeSlider(name = "Rush Yards", start = 0, end = 150),
       rbRushTds = pn.widgets.RangeSlider(name = "Rush Touchdowns", start = 0, end = 5),
       rbFum = pn.widgets.RangeSlider(name = "Fumbles", start = 0, end = 1)
           )

Column
    [0] Column
        [0] RangeSlider(end=150, name='Rush Yards', step=1, value=(0, 150))
        [1] RangeSlider(end=5, name='Rush Touchdowns', step=1, value=(0, 5))
        [2] RangeSlider(name='Fumbles', step=1)
    [1] Row
        [0] Column
            [0] Row
                [0] HoloViews(Bars)
                [1] HoloViews(Bars)
            [1] Row
                [0] HoloViews(Scatter)
                [1] Plotly(Figure)